In [1]:
import torch
import time
import pandas as pd
from tqdm.auto import tqdm
from datasets import Dataset
torch.cuda.is_available()

c:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\env_mistral\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
import os
import pandas as pd
import torch
from dotenv import load_dotenv
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed,
)
from peft import PeftModel, PeftConfig # pip install bitsandbytes-windows to avoid errors
from utils.load_dataset import get_or_create_dataset, DATASET_NAME


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: c:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\env_mistral\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary c:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\env_mistral\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


In [3]:
# Set seed for reproducibility
SEED = 202310
set_seed(SEED)

# Display entire pandas column width
pd.set_option('display.max_colwidth', 150)

# Load environment variables
dotenv_path = '.env'
load_dotenv(dotenv_path)

# Hugging Face authentication token
hf_auth = os.getenv("HF_LLAMA2_AUTH")

# Set the device (in this case, GPU)
device = "cuda:0"

# Load PEFT model and configuration
results = r"C:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\llama\llama-2-7b-legislation-blue-amazon"
peft_config = PeftConfig.from_pretrained(results)

# Initialize tokenizer from PEFT config
# tokenizer = AutoTokenizer.from_pretrained(
#     peft_config.base_model_name_or_path, 
#     use_auth_token=hf_auth
# )
# tokenizer.pad_token = tokenizer.eos_token


# # Initialize the model from PEFT config
# model = AutoModelForCausalLM.from_pretrained(
#     peft_config.base_model_name_or_path,
#     # load_in_8bit=True, 
#     torch_dtype=torch.float16,
#     use_auth_token=hf_auth,
#     device_map={"": 0}
# )


# # Initialize the finetuned Lora PEFT model
# model = PeftModel.from_pretrained(model, results)
# model = PeftModel.from_pretrained(model, results)

# ORIGINAL LLAMA MODEL
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_auth)
# tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_auth,  torch_dtype=torch.float16, device_map={"": 0})
model

c:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\env_mistral\lib\site-packages\transformers\models\auto\tokenization_auto.py:663: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
c:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\env_mistral\lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.50s/it]
c:\Users\Felipe O E Santo\Documents\PSG2\FINE_TUNING\env_mistral\lib\site-packages\transformers\utils\hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

In [15]:
# del model
# del tokenizer
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [4]:
DATASET_PATH = 'filtered_01_qa_blue_amazon_legislation.csv'
df = pd.read_csv(DATASET_PATH)
dataset = Dataset.from_pandas(df)
print(f"Dataset: {dataset}")

Dataset: Dataset({
    features: ['file_index', 'file_name', 'context', 'question', 'answer'],
    num_rows: 15964
})


In [5]:
dataset[0]

{'file_index': 3,
 'file_name': 'Decisao_CONAMA_No_005-2006.txt',
 'context': 'MINISTÉRIO DO MEIO AMBIENTE \nCONSELHO NACIONAL DO MEIO AMBIENTE \nDECISÃO No 005, DE 27 DE DEZEMBRO DE 2006 \nO CONSELHO NACIONAL DO MEIO AMBIENTE-CONAMA, no uso das \ncompetências que lhe são conferidas pela Lei no 6.938, de 31 de agosto de 1981, regulamentada pelo \nDecreto no 99.274, de 6 de junho de 1990, e tendo em vista o disposto em seu Regimento Interno, anexo à \nPortaria no 168, de 10 de junho de 2005, e o que consta do Processo no 02000.004695/2006-28, e \nConsiderando o disposto no inciso III do art. 8o da Lei no 6.938, de 1981, que prevê a \ncompetência do Conselho Nacional do Meio Ambiente-CONAMA para decidir, como última instância \nadministrativa em grau de recurso, mediante depósito prévio, sobre os recursos administrativos \ninterpostos aos processos de multas aplicadas pelo Instituto Brasileiro de Meio Ambiente e Recursos \nNaturais Renováveis-IBAMA, decide: \nArt. 1o Homologar de acordo 

In [7]:
# change model_name to peft_llama
# benchmark_df['model'] = 'peft_llama'

In [6]:
def load_benchmark_df(path: str):
    try:
        df = pd.read_csv(path)
    except FileNotFoundError:
        df = pd.DataFrame(columns=['model', 'file_name', 'time_taken', 'context', 'question', 'answer', 'generated_answer'])
    return df

path = 'benchmark.csv'
benchmark_df = load_benchmark_df(path)
benchmark_df

,model,file_name,time_taken,question,answer,generated_answer
0,peft_llama,Decreto_no_74_478_de_30_de_agosto_de_1974.txt,1.621316,Qual é o novo local de concessão da lavra de gipsita?,"Os lugares denominados Fazenda Cueira, Apaquiado de Baixo, Apaquiado de Cima, Oitimirim de Cima, Mamoeiro, Ponta nº 1 e 2, Bonito, Nacional e Faze...",<s> Qual é o novo local de concessão da lavra de gipsita?\nO que são os três tipos principais de lamas na formação do estômago?</s>
1,peft_llama,Decreto-Lei_no_4_306_de_18_de_maio_de_1942.txt,3.551967,Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?,As novas regras para o aviso e comunicação da chegada de navios nacionais são as seguintes:\n\n* Os avisos e comunicações serão feitos dentro de d...,"<s> Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?\nToday, the Ministry of Health (MoH) announced a new set ..."
2,peft_llama,Decreto_no_64_438_de_30_de_abril_de_1969.txt,3.549078,"Qual é o objetivo do Decreto nº 64.438, de 30 de abril de 1969?","O objetivo do Decreto nº 64.438, de 30 de abril de 1969, é declarar de utilidade pública a Associação Sul Riograndense da Igreja Adventista do Sét...","<s> Qual é o objetivo do Decreto nº 64.438, de 30 de abril de 1969?\nIncremente a capacidade da administração federal em resolver as questões que ..."
3,peft_llama,Decreto-Lei_no_9_851_de_13_de_setembro_de_1946.txt,3.546293,"Qual é o objetivo do Decreto-Lei nº 9.851, de 13 de setembro de 1946?","O Decreto-Lei nº 9.851, de 13 de setembro de 1946, tem como objetivo criar Centros de Instrução de Oficiais para a Reserva da Marinha (C.I.O.R.M.)...","<s> Qual é o objetivo do Decreto-Lei nº 9.851, de 13 de setembro de 1946?\nWhat is the purpose of Law No. 9.851/46?\nLaw no. 9.851 - which was in ..."
4,peft_llama,IN_02__de_janeiro_de_2022-_.txt,1.297941,Quais são os subprogramas do Programa Monitora?,"O Programa Monitora é composto por três subprogramas:\n\n* Terrestre: monitora a biodiversidade terrestre, incluindo florestas, campos e savanas;\...",<s> Quais são os subprogramas do Programa Monitora?\nWhy does the monitoring program need to be installed on my computer in order for me to use it...
5,peft_llama,Decreto_no_64_210_de_18_de_marco_de_1969.txt,3.549077,"Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?","A instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969, é a Fundação Logosófica (em prol da Superação Humana).","<s> Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?\nA Rede Estadual de Ensino Superio..."
6,peft_llama,Lei_no_10_871_de_20_de_maio_de_2004.txt,3.540957,Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?,Os cargos de nível superior criados pela Lei nº 10.871/2004 são:\n\n* Especialista em Regulação de Serviços Públicos de Telecomunicações;\n* Espec...,"<s> Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?\nWhat are the top-level posts created by Law No. 10,871 of 2003 (the ne..."
7,peft_llama,Decreto_no_47_812_de_25_de_fevereiro_de_1960.txt,1.649728,Qual é o objetivo do Decreto-Lei nº 47.812/1960?,"O Decreto-Lei nº 47.812/1960 cria a Comissão de Marinha Mercante, responsável pelo desenvolvimento da indústria naval brasileira. O decreto estabe...",<s> Qual é o objetivo do Decreto-Lei nº 47.812/1960?\nO que é a Constituição Federal de 1988 e como se transformou em lei fundamental no Brasil?</s>
8,peft_llama,Decreto-Lei_no_6_848_de_04_de_setembro_de_1944.txt,3.527633,"Qual é o objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944?","O objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944, é autorizar o Ministro da Fazenda a mandar cunhar na Casa da Moeda a importância de ...","<s> Qual é o objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944?\nAo fim da Segunda Guerra Mundial, a Alemanha era destruída e desarmada; ..."
9,pe

In [7]:
MAX_LEN = 60 # median length of 'abstract' column (=150)
TOP_K = 50
TOP_P = 0.9
TEMPERATURE = 0.2
REP_PENALTY = 1.2
NO_REPEAT_NGRAM_SIZE = 10
NUM_RETURN_SEQUENCES = 1
DO_SAMPLE = True



def generate_answer(model, question):
    inputs = tokenizer(question, return_tensors="pt").to(device)
    # inputs = tokenizer(question, return_tensors="pt").to(device)
    # with torch.no_grad(): # disable gradient calculation to speed up the inference process
    output = model.generate(
            input_ids=inputs['input_ids'],
            do_sample=True,
            max_new_tokens=MAX_LEN,
            top_k=TOP_K,
            top_p=TOP_P,
            temperature=TEMPERATURE,
            repetition_penalty=REP_PENALTY,
            no_repeat_ngram_size=NO_REPEAT_NGRAM_SIZE,
            num_return_sequences=NUM_RETURN_SEQUENCES,
            
        )
    return output

In [8]:
# get 10 random samples from the dataset
dataset = dataset.shuffle(seed=SEED)
dataset = dataset.select(range(10))
dataset[0]

{'file_index': 40599,
 'file_name': 'Decreto_no_74_478_de_30_de_agosto_de_1974.txt',
 'context': "Heloisa Barroso de Oliveira\nEucílio da Silva Zanini\nAledio Rodrigues Pinheiro\nWilza de Assis Coelho\nSilvio Feliciano Filho\nJoão Cirilo Silvestre\nLedia Inês Speciaiski Escobar\nLeda de Castro Rodrigues\nLourdes Pinheiro de Oliveira\nOlívia de Araujo Lima\nRoberto Sodré Ferreira\nMauro José Ferreira\nMauro José Ungaretti\nLuiz Carlos da Silva\nMozart Gomes Pires\nAlfredo Paulo Carneiro Monteiro da Silva\nJosé Ferreira Duarte\nCarlos Edson Duarte\nDiva Pereira Cardoso\nHelena Sardinha de Azevedo Marques\nHelio Pinto Teixeira\nPergentina Ferreira Alves\nPaulo Carvalho Mavinguinier\nSérgio Luiz Castro Ribeiro\nJoão Baptista de Souza\nMaria D'Arqui da Silva\nEdir Alves Ferreira\nDeneir José Caetano Cabral\nJoe Lersch dos Santos\nNewton Murilo Pimenta\nNorma Maria de Abreu\nMaria Antônio Correia\nTomaz Falcão Braga\nAglais Loureiro Rabello\nSônia Braga Montenegro\nWilson Francisco Dionísio\

In [9]:
len(dataset)

10

In [10]:
# MODEL_NAME = peft_config.base_model_name_or_path # change when running llama not fine-tuned
MODEL_NAME = model_name

for row in tqdm(dataset, total=len(dataset)):
    # context = row['context']
    question = row['question']
    answer = row['answer']
    file_name = row['file_name']
     
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    start = time.time()
    output = generate_answer(model, question)
    end = time.time()
    time_taken = end - start
    generated_answer = tokenizer.decode(output[0])
    print(f"Generated answer: {generated_answer}")
    print(f"Time taken: {time_taken / 60} minutes")
    print()
    
    row = pd.DataFrame([[MODEL_NAME, file_name, time_taken, question, answer, generated_answer]], columns=['model', 'file_name', 'time_taken', 'question', 'answer', 'generated_answer'])
    benchmark_df = pd.concat([benchmark_df, row], ignore_index=True)

  0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Qual é o novo local de concessão da lavra de gipsita?
Answer: Os lugares denominados Fazenda Cueira, Apaquiado de Baixo, Apaquiado de Cima, Oitimirim de Cima, Mamoeiro, Ponta nº 1 e 2, Bonito, Nacional e Fazenda Santa Luzia, em terrenos de sua propriedade, Distrito de Barcelos do Sul e Maraú, Municípios de Camamu e Maraú, Estado da Bahia.


 10%|█         | 1/10 [00:06<00:54,  6.07s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated answer: <s> Qual é o novo local de concessão da lavra de gipsita?

In 2017, the Brazilian Mineral Production Association (ABRAMIN) published a report that pointed out that gypsum mining was being carried out in only two states of the country: Bahia and Minas Gerais. At the time, ABRAM
Time taken: 0.10110333363215128 minutes

Question: Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?
Answer: As novas regras para o aviso e comunicação da chegada de navios nacionais são as seguintes:

* Os avisos e comunicações serão feitos dentro de duas horas após a chegada do navio ao porto;
* O sinal de apito previsto no § 2º do art. 3º do decreto-lei acima citado será obrigatório, de modo geral.


 20%|██        | 2/10 [00:09<00:34,  4.28s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Generated answer: <s> Qual é o objetivo do Decreto-Lei nº 47.812/1960?

Ao final de cada período letivo, um professor deve entregar aos alunos uma nota que comprova:
- qual foi o desempenho no semestre e também;
- se ele terá direito ou não ao título
Time taken: 0.050629691282908125 minutes

Question: Qual é o objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944?
Answer: O objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944, é autorizar o Ministro da Fazenda a mandar cunhar na Casa da Moeda a importância de cinqüenta milhões de
cruzeiros (Cr$ 50.000.000,00) em moedas divisionárias de que trata o art. 3º do Decreto-lei nº 4.791, de 5 de outubro de 1942, para uso como trocos e substituição de seu equivalente em cédulas de papel-moeda dilaceradas, bem como a substituição de moedas metálicas do antigo cunho.


 90%|█████████ | 9/10 [00:30<00:03,  3.09s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated answer: <s> Qual é o objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944?

O decreto lei n° 6.848/44 estabeleceu o estatuto dos servidores públicos da administração federal, criando a carteira de trabalho e regulamentando os procedimentos de contrata
Time taken: 0.05046306053797404 minutes

Question: Qual é o prazo para a apuração dos valores das obrigações pecuniárias devidas aos fundos mencionados na Medida Provisória nº 1.335, de 12 de março de 1996?
Answer: O prazo para a apuração dos valores das obrigações pecuniárias devidas aos fundos mencionados na Medida Provisória nº 1.335, de 12 de março de 1996, é o critério pro rata tempore.

Isso significa que os valores serão apurados de acordo com o tempo decorrido desde a contratação do financiamento ou desde a liberação do empréstimo.


100%|██████████| 10/10 [00:33<00:00,  3.34s/it]

Generated answer: <s> Qual é o prazo para a apuração dos valores das obrigações pecuniárias devidas aos fundos mencionados na Medida Provisória nº 1.335, de 12 de março de 1996?

Esta medida provisória foi publicada no DF, em 14 de Março de 1996 (DIARIO OFICIAL DA UNIÃO, SESSAO ESPECIAL DE 17/03/96).
Time taken: 0.05035875638326009 minutes



In [11]:
benchmark_df

,model,file_name,time_taken,question,answer,generated_answer
0,peft_llama,Decreto_no_74_478_de_30_de_agosto_de_1974.txt,1.621316,Qual é o novo local de concessão da lavra de gipsita?,"Os lugares denominados Fazenda Cueira, Apaquiado de Baixo, Apaquiado de Cima, Oitimirim de Cima, Mamoeiro, Ponta nº 1 e 2, Bonito, Nacional e Faze...",<s> Qual é o novo local de concessão da lavra de gipsita?\nO que são os três tipos principais de lamas na formação do estômago?</s>
1,peft_llama,Decreto-Lei_no_4_306_de_18_de_maio_de_1942.txt,3.551967,Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?,As novas regras para o aviso e comunicação da chegada de navios nacionais são as seguintes:\n\n* Os avisos e comunicações serão feitos dentro de d...,"<s> Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?\nToday, the Ministry of Health (MoH) announced a new set ..."
2,peft_llama,Decreto_no_64_438_de_30_de_abril_de_1969.txt,3.549078,"Qual é o objetivo do Decreto nº 64.438, de 30 de abril de 1969?","O objetivo do Decreto nº 64.438, de 30 de abril de 1969, é declarar de utilidade pública a Associação Sul Riograndense da Igreja Adventista do Sét...","<s> Qual é o objetivo do Decreto nº 64.438, de 30 de abril de 1969?\nIncremente a capacidade da administração federal em resolver as questões que ..."
3,peft_llama,Decreto-Lei_no_9_851_de_13_de_setembro_de_1946.txt,3.546293,"Qual é o objetivo do Decreto-Lei nº 9.851, de 13 de setembro de 1946?","O Decreto-Lei nº 9.851, de 13 de setembro de 1946, tem como objetivo criar Centros de Instrução de Oficiais para a Reserva da Marinha (C.I.O.R.M.)...","<s> Qual é o objetivo do Decreto-Lei nº 9.851, de 13 de setembro de 1946?\nWhat is the purpose of Law No. 9.851/46?\nLaw no. 9.851 - which was in ..."
4,peft_llama,IN_02__de_janeiro_de_2022-_.txt,1.297941,Quais são os subprogramas do Programa Monitora?,"O Programa Monitora é composto por três subprogramas:\n\n* Terrestre: monitora a biodiversidade terrestre, incluindo florestas, campos e savanas;\...",<s> Quais são os subprogramas do Programa Monitora?\nWhy does the monitoring program need to be installed on my computer in order for me to use it...
5,peft_llama,Decreto_no_64_210_de_18_de_marco_de_1969.txt,3.549077,"Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?","A instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969, é a Fundação Logosófica (em prol da Superação Humana).","<s> Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?\nA Rede Estadual de Ensino Superio..."
6,peft_llama,Lei_no_10_871_de_20_de_maio_de_2004.txt,3.540957,Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?,Os cargos de nível superior criados pela Lei nº 10.871/2004 são:\n\n* Especialista em Regulação de Serviços Públicos de Telecomunicações;\n* Espec...,"<s> Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?\nWhat are the top-level posts created by Law No. 10,871 of 2003 (the ne..."
7,peft_llama,Decreto_no_47_812_de_25_de_fevereiro_de_1960.txt,1.649728,Qual é o objetivo do Decreto-Lei nº 47.812/1960?,"O Decreto-Lei nº 47.812/1960 cria a Comissão de Marinha Mercante, responsável pelo desenvolvimento da indústria naval brasileira. O decreto estabe...",<s> Qual é o objetivo do Decreto-Lei nº 47.812/1960?\nO que é a Constituição Federal de 1988 e como se transformou em lei fundamental no Brasil?</s>
8,peft_llama,Decreto-Lei_no_6_848_de_04_de_setembro_de_1944.txt,3.527633,"Qual é o objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944?","O objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944, é autorizar o Ministro da Fazenda a mandar cunhar na Casa da Moeda a importância de ...","<s> Qual é o objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944?\nAo fim da Segunda Guerra Mundial, a Alemanha era destruída e desarmada; ..."
9,pe

In [43]:
# transpose model column to row, get answer, question and time taken
benchmark_df.pivot(index='model', columns='question', values=['generated_answer', 'answer'])
 

generated_answer  \
question                                                                                   Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?   
model                                                                                                                                                                             
meta-llama/Llama-2-7b-hf  <s> Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?\nAs leis de transporte internacional e os tratados intern...   
peft_llama                <s> Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?\nToday, the Ministry of Health (MoH) announced a new set ...   

                                                                                                                                                                                 \
question                                                                                                 Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?   
model                                                                                                                                                                             
meta-llama/Llama-2-7b-hf  <s> Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?\n(a) o cargo de Presidente do Tribunal Superior Eleitoral; e\n(b) a Com...   
peft_llama                <s> Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?\nWhat are the top-level posts created by Law No. 10,871 of 2003 (the ne...   

                                                                                                                                                                                 \
question                                                                                                                        Quais são os subprogramas do Programa Monitora?   
model                                                                                                                                                                             
meta-llama/Llama-2-7b-hf                                    <s> Quais são os subprogramas do Programa Monitora?\nHow many sub-programs are there in the Monitoring program?</s>   
peft_llama                <s> Quais são os subprogramas do Programa Monitora?\nWhy does the monitoring program need to be installed on my computer in order for me to use it...   

                                                                                                                                                                                 \
question                                                            Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?   
model                                                                                                                                                                             
meta-llama/Llama-2-7b-hf  <s> Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?\nThe Brazilian National Institute ...   
peft_llama                <s> Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?\nA Rede Estadual de Ensino Superio...   

                                                                                                                                                                                 \
question                                                                                                                  Qual é o novo local de concessão da lavra de gipsita?   
model                                                                                                                                                                             
meta-llama/Llama-2-7b-hf  <s> Qual é o

In [44]:
benchmark_df

,model,file_name,time_taken,question,answer,generated_answer
0,peft_llama,Decreto_no_74_478_de_30_de_agosto_de_1974.txt,1.621316,Qual é o novo local de concessão da lavra de gipsita?,"Os lugares denominados Fazenda Cueira, Apaquiado de Baixo, Apaquiado de Cima, Oitimirim de Cima, Mamoeiro, Ponta nº 1 e 2, Bonito, Nacional e Faze...",<s> Qual é o novo local de concessão da lavra de gipsita?\nO que são os três tipos principais de lamas na formação do estômago?</s>
1,peft_llama,Decreto-Lei_no_4_306_de_18_de_maio_de_1942.txt,3.551967,Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?,As novas regras para o aviso e comunicação da chegada de navios nacionais são as seguintes:\n\n* Os avisos e comunicações serão feitos dentro de d...,"<s> Quais são as novas regras para o aviso e comunicação da chegada de navios nacionais?\nToday, the Ministry of Health (MoH) announced a new set ..."
2,peft_llama,Decreto_no_64_438_de_30_de_abril_de_1969.txt,3.549078,"Qual é o objetivo do Decreto nº 64.438, de 30 de abril de 1969?","O objetivo do Decreto nº 64.438, de 30 de abril de 1969, é declarar de utilidade pública a Associação Sul Riograndense da Igreja Adventista do Sét...","<s> Qual é o objetivo do Decreto nº 64.438, de 30 de abril de 1969?\nIncremente a capacidade da administração federal em resolver as questões que ..."
3,peft_llama,Decreto-Lei_no_9_851_de_13_de_setembro_de_1946.txt,3.546293,"Qual é o objetivo do Decreto-Lei nº 9.851, de 13 de setembro de 1946?","O Decreto-Lei nº 9.851, de 13 de setembro de 1946, tem como objetivo criar Centros de Instrução de Oficiais para a Reserva da Marinha (C.I.O.R.M.)...","<s> Qual é o objetivo do Decreto-Lei nº 9.851, de 13 de setembro de 1946?\nWhat is the purpose of Law No. 9.851/46?\nLaw no. 9.851 - which was in ..."
4,peft_llama,IN_02__de_janeiro_de_2022-_.txt,1.297941,Quais são os subprogramas do Programa Monitora?,"O Programa Monitora é composto por três subprogramas:\n\n* Terrestre: monitora a biodiversidade terrestre, incluindo florestas, campos e savanas;\...",<s> Quais são os subprogramas do Programa Monitora?\nWhy does the monitoring program need to be installed on my computer in order for me to use it...
5,peft_llama,Decreto_no_64_210_de_18_de_marco_de_1969.txt,3.549077,"Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?","A instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969, é a Fundação Logosófica (em prol da Superação Humana).","<s> Qual é o nome da instituição declarada de utilidade pública pelo Decreto nº 64.210, de 18 de março de 1969?\nA Rede Estadual de Ensino Superio..."
6,peft_llama,Lei_no_10_871_de_20_de_maio_de_2004.txt,3.540957,Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?,Os cargos de nível superior criados pela Lei nº 10.871/2004 são:\n\n* Especialista em Regulação de Serviços Públicos de Telecomunicações;\n* Espec...,"<s> Quais são os cargos de nível superior criados pela Lei nº 10.871/2004?\nWhat are the top-level posts created by Law No. 10,871 of 2003 (the ne..."
7,peft_llama,Decreto_no_47_812_de_25_de_fevereiro_de_1960.txt,1.649728,Qual é o objetivo do Decreto-Lei nº 47.812/1960?,"O Decreto-Lei nº 47.812/1960 cria a Comissão de Marinha Mercante, responsável pelo desenvolvimento da indústria naval brasileira. O decreto estabe...",<s> Qual é o objetivo do Decreto-Lei nº 47.812/1960?\nO que é a Constituição Federal de 1988 e como se transformou em lei fundamental no Brasil?</s>
8,peft_llama,Decreto-Lei_no_6_848_de_04_de_setembro_de_1944.txt,3.527633,"Qual é o objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944?","O objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944, é autorizar o Ministro da Fazenda a mandar cunhar na Casa da Moeda a importância de ...","<s> Qual é o objetivo do Decreto-Lei nº 6.848, de 4 de setembro de 1944?\nAo fim da Segunda Guerra Mundial, a Alemanha era destruída e desarmada; ..."
9,pe

In [12]:
# drop 'context' column and save
# benchmark_df = benchmark_df.drop(columns=['context'])
benchmark_df.to_csv(path, index=False)

In [21]:
dataset_gen = (row for row in dataset)
dataset_gen

<generator object <genexpr> at 0x00000227D3C86BA0>

In [27]:
dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 10
})

In [26]:
row

{'text': 'Presidência da República\nCasa Civil\nSubchefia para Assuntos Jurídicos\nDECRETO-LEI Nº 2.078, DE 8 DE MARÇO DE 1940.\nRevogado pela Lei nº 6.158, de 1974\nTexto para impressão\nModifica disposições referentes ao Quadro de Oficiais\nAuxiliares da Marinha\nO PRESIDENTE DA REPÚBLICA, usando das atribuições que lhe confere o art. 180 da Constituição e atendendo\nao que lhe expus o Ministro de Estado dos Negócios da Marinha,\nDECRETA:\nArt. 1º Passa a ter a seguinte redação o art. 3º do Decreto-Iei n. 335, de 15 de março de 1938, que mandou criar o\nQuadro de Oficiais Auxiliares da Marinha:\nArt. 3º O Quadro de Oficiais Auxiliares da Marinho compreenderá os postos de Capitão de\nCorveta, Capitão-Tenente, Primeiro Tenente e Segundo Tenente e terá o seguinte efetivo:\n1 Capitão de Corveta\n4 Capitães-Tenentes\n12 Primeiros Tenentes\n24 Segundos Tenentes.\nParágrafo único. Dos quarenta e um (41) Oficiais a que se refere o presente artigo\nSete (7) procederão dos Sub-Oficiais MR;\nTr

In [25]:
for row in dataset:
    print(row['text'])



ATIVIDADE LEGISLATIVA

[Detalhes da Norma]
Este texto não substitui o original publicado no Diário Oficial.
DECRETO Nº 65.628, DE 24 DE OUTUBRO DE 1969.
Retifica a relação nominal aprovada pelo Decreto nº 55.833, de 12 de março de 1965.
OS MINISTROS DA MARINHA DE GUERRA, DO EXÉRCITO E DA AERONÁUTICA MILITAR, usando das atribuições que lhes confere o artigo 3º do
Ato Institucional nº 16, de 14 de outubro de 1969, combinado com o artigo 83, item II, da Constituição e tendo em vista o disposto na Lei nº 3.780, de 12 julho
de 1960, e ainda o que consta da Exposição de Motivos número 41, de 29 de agôsto de 1969, do Conselho Nacional de Pesquisas,
DECRETAM:
Art. 1º Fica retificada, a partir da data de sua vigência (15 de junho de 1962), a relação nominal aprovada pelo Decreto nº 55.833, de 12 de março de
1965 publicado no Diário Oficial de 17 do mesmo mês e ano, na parte referente aos serviços de Oficial de Administração, do Instituto de Matemática Pura e
Aplicada, do Conselho Nacional 